<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangChain(1)_BasicTransaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 LangChain とは何か：LLM 開発の「無秩序」を防ぐための標準規格

## 1. API を叩くだけなら LangChain は要らない
まず誤解を恐れずに言えば、「ただ AI とチャットするだけの短いスクリプト」を書くなら、LangChain は不要です。OpenAI や DeepSeek の API を直接 `requests` や公式ライブラリで叩けば、わずか数行で動きます。

しかし、**「プロトタイプ（試作品）」を「プロダクト（実用的なシステム）」にしようとした瞬間、開発の難易度は跳ね上がります。**

LLM（大規模言語モデル）は、従来のソフトウェア部品とは異なり、**「確率的で、曖昧で、ステートレス（記憶を持たない）」** という扱いづらい性質を持っています。この扱いづらい LLM を、システムの中に信頼できる部品として組み込むためのフレームワークが **LangChain** です。

---

## 2. なぜ LangChain が必要なのか？

LangChain を学ばずに、Python の標準機能だけで LLM アプリを作り始めた場合、遅かれ早かれ以下の **3つの「技術的負債」** に直面します。これこそが「LangChain を学んでおかないとまずい理由」です。

### ① 「文字列操作スパゲッティ」の壁（保守性の欠如）
LLM への入力はすべて「テキスト（プロンプト）」です。
最初は Python の f-string（文字列埋め込み）で `f"あなたは{role}です。次の質問に答えて：{question}"` と書けば済みます。しかし、機能が増えるとコードは複雑化します。

* 「過去の会話履歴を5往復分だけ入れたい」
* 「ユーザーが英語で質問してきたら、英語用の指示に変えたい」
* 「検索結果が空だった場合は、別のプロンプトを使いたい」

これらを `if` 文と文字列結合だけで管理しようとすると、コードは瞬く間に解読不能な状態になります。LangChain は、プロンプトを単なる文字列ではなく **「管理可能なオブジェクト（部品）」** として扱い、きれいに結合する仕組みを提供します。

### ② 「モデル依存」の壁（ベンダーロックイン）
「OpenAI の GPT-5.2 でアプリを作りました。完璧です」
数ヶ月後、上司やクライアントが言います。「DeepSeek R1 の方が安くて速いらしいから、そっちに変えて。あと、特定の処理だけは Claude 4.5 を使って。」

LangChain を使っていない場合、あなたはコード内の **API 通信部分、パラメータ設定、エラー処理、レスポンスの受け取り処理をすべて書き直す** ことになります。各社で微妙に仕様が異なるからです。
LangChain は、これらを共通のインターフェース（`ChatModel`）でラップしているため、たった1行、モデル名を書き換えるだけで移行が完了します。

### ③ 「状態管理」の壁（ステートレスへの対応）
HTTP 通信も LLM も、基本的には「ステートレス（前のことを覚えていない）」です。
「今の発言」をするためには、「過去の会話履歴」をすべて毎回送信し直す必要があります。

* 会話が長くなったら、古い順に捨てるのか？
* 要約して圧縮するのか？
* データベースに保存して、必要なときだけ取り出すのか？

これをゼロから実装するのは非常にバグを生みやすい作業です。LangChain は、この **「記憶（Memory）」の管理** を標準機能として提供しており、データベースとの連携も容易です。

---

## 3. 結論：LangChain は「LLM アプリの OS」
LangChain を学ぶということは、単に便利なライブラリの使い方を覚えるということではありません。

* **入力（Prompt）**
* **処理（Model / Chain）**
* **記憶（Memory）**
* **出力（Output Parser）**

これらをどのように設計・接続すべきかという **「LLM アプリケーションのアーキテクチャ（設計思想）」** を学ぶことと同義です。

本コースでは、便利な機能を表面的になぞるのではなく、**「LangChain が裏側で何を解決してくれているのか」** をコードレベルで理解することを目指します。

### 🔑 API キーの設定

Colab の「シークレット（鍵マーク）」に登録した `OPENROUTER_API_KEY` を読み込みます。

In [ ]:
!pip install -qU langchain-openai langchain-core termcolor

import os
from google.colab import userdata
from termcolor import cprint

try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    cprint("✅ API Key loaded.", "green")
except Exception as e:
    cprint("❌ Error: API Key not found in Colab secrets.", "red")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 15.8 MB/s eta 0:00:00
✅ API Key loaded.


### 1️⃣ LangChain でのモデル呼び出し

LangChain では、OpenAI や OpenRouter などの異なるサービスを共通の方法で扱えるようになっています。
ここでは `ChatOpenAI` クラスを使用し、OpenRouter 上の `deepseek/deepseek-chat-v3-0324` モデルに接続します。

まずは最も単純な、文字列を渡して文字列を受け取る方法を確認します。

In [ ]:
from langchain_openai import ChatOpenAI

# モデルの初期化
# base_url を指定することで、OpenAI ではなく OpenRouter に接続します
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7
)

# 単純なテキストを送信して実行 (invoke)
response = llm.invoke("Pythonとは何ですか？一言で説明してください。")

# 結果の表示
print(f"▼ 返答の型: {type(response)}")
cprint(f"▼ 返答内容:\n{response.content}", "cyan")

### 2️⃣ ロール（役割）の指定

LLM との対話では、単なるテキストではなく **「誰が発言したか（Role）」** という情報が重要になります。
LangChain では、これを専用のクラスで管理します。

* **SystemMessage:** AI の振る舞いや設定（例：「あなたは数学の先生です」）。
* **HumanMessage:** ユーザーからの入力。

これらをリスト（配列）にして渡すことで、文脈を持った対話が可能になります。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# メッセージリストの作成
messages = [
    SystemMessage(content="あなたは河村たかし氏のように名古屋弁を話す気軽なアシスタントです。"),
    HumanMessage(content="あつた蓬莱軒で食べようと思ったら満席だった。")
]

# リストを渡して実行
response_with_role = llm.invoke(messages)

cprint(f"▼ 静岡弁での返答:\n{response_with_role.content}", "cyan")

### 3️⃣ 内部データ構造の確認（課題）

LangChain で `SystemMessage` や `HumanMessage` を使って記述しましたが、**通信のレベルではこれらは単なる辞書（JSON形式のテキストデータ）** です。

以下のコードを実行して、LangChain のオブジェクトが最終的にどのようなデータ形式で扱われているかを確認してください。これが LLM が理解している「実体」です。

In [ ]:
# LangChainのメッセージオブジェクトの中身を確認する
print("--- LangChain Objects ---")
print(f"SystemMessage: {messages[0]}")
print(f"HumanMessage:  {messages[1]}")

print("\n--- Equivalent Raw Data (JSON format) ---")
# 実際にAPIに送られるときは、以下のような辞書形式（プレインテキスト）になります
raw_data = [
    {"role": "system", "content": messages[0].content},
    {"role": "user",   "content": messages[1].content}
]

import json
print(json.dumps(raw_data, indent=2, ensure_ascii=False))

# 【課題】
# 上記の `raw_data` の構造を理解した上で、
# messages リストにもう一つ HumanMessage を追加して、
# 「Pythonで "Hello" と表示するコードを書いて」と依頼するコードを書いて、実行してください。
# 追加後の messages を llm.invoke(messages) に渡します。

# ↓ ここにコードを記述してください

### 4️⃣ `invoke` メソッドの正体

ここまで当たり前のように `llm.invoke(...)` を使ってきましたが、この `invoke` とは何でしょうか？

LangChain における `invoke` は、**「入力（Input）を受け取り、処理を実行し、出力（Output）を返す」** という命令です。
LLM オブジェクトに対して `invoke` を実行すると、以下のことが行われます。

1.  **送信:** 引数で渡されたメッセージ（リストや文字列）を API サーバーへ送信する。
2.  **待機:** サーバーでの計算が終わり、レスポンスが返ってくるまでプログラムの実行を停止して待つ（同期処理）。
3.  **ラップ:** 返ってきた JSON データを `AIMessage` という扱いやすいオブジェクトに変換して返す。

単に文字列を返しているのではなく、**「メタデータ（付加情報）を含んだメッセージオブジェクト」** を返している点に注目してください。

以下の課題で、その戻り値を詳しく見てみましょう。

In [ ]:
from langchain_core.messages import HumanMessage

# シンプルなメッセージを作成
message = [HumanMessage(content="1+1は？")]

print("--- invoke 実行開始 ---")

# invokeを実行し、その戻り値をそのまま変数に格納
result_object = llm.invoke(message)

print("--- invoke 実行終了 ---")

# 【課題】 invoke の戻り値を詳しく確認
# 以下の print 文の結果を見て、invoke が単なる文字列("2")ではなく、
# 情報を包んだオブジェクトを返していることを確認してください。

print(f"1. 戻り値の型 (Type): {type(result_object)}")
print(f"2. オブジェクトの中身: {result_object}")
print(f"3. コンテンツの抽出: {result_object.content}")
print(f"4. API利用状況 (Metadata): {result_object.response_metadata}")

# ※ response_metadata には、トークン使用量やモデル名などが含まれています。
# これらは invoke が API からの生のレスポンスを解析して格納したものです。